In [20]:
import fiftyone as fo
from utils import dataloader


data_train = dataloader.load_coco_dataset('train')
data_train

hi
Found annotations at '/home/zach/fiftyone/coco-2017/raw/instances_train2017.json'
Sufficient images already downloaded
Existing download of split 'train' is sufficient
Loading existing dataset 'coco-2017-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Name:        coco-2017-train
Media type:  image
Num samples: 64115
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [19]:
%load_ext autoreload
%autoreload 2

In [2]:
session = fo.launch_app(data_train)

In [3]:
session

Dataset:          coco-2017-train
Media type:       image
Num samples:      64115
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

In [4]:
data_val = dataloader.load_coco_dataset('validation')

Found annotations at '/home/zach/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [5]:
session = fo.launch_app(data_val)

In [6]:
session.close()

In [7]:
session

Dataset:          coco-2017-validation
Media type:       image
Num samples:      2693
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

In [8]:
data_train

Name:        coco-2017-train
Media type:  image
Num samples: 64115
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [9]:
x = data_train.take(10,seed=50)

In [10]:
x

Dataset:     coco-2017-train
Media type:  image
Num samples: 10
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    1. Take(size=10, seed=50)

In [11]:
y = x.values('filepath')

In [12]:
y

['/home/zach/fiftyone/coco-2017/train/data/000000530631.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000418315.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000088458.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000554621.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000350171.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000242246.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000443094.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000518761.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000028116.jpg',
 '/home/zach/fiftyone/coco-2017/train/data/000000285967.jpg']

In [16]:
sample = x[y]
sample

InvalidId: '/home/zach/fiftyone/coco-2017/train/data/000000530631.jpg' is not a valid ObjectId, it must be a 12-byte input or a 24-character hex string

In [14]:
len(sample['ground_truth'].detections)

InvalidId: '/home/zach/fiftyone/coco-2017/train/data/000000530631.jpg' is not a valid ObjectId, it must be a 12-byte input or a 24-character hex string

In [ ]:
sample

In [21]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model(num_classes):
    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model